## Một cách tiếp cận tính toán áp dụng Simulated Annealing để nghiên cứu tính ổn định của mạng tương tác protein trong ung thư và rối loạn thần kinh. 

Authors: NhanTV, Trung NT <br>
Master Student<br>
VNU - HUS


## Mục đích

Dựng lại bài báo khoa học: "A computational approach inspired by simulated annealing to study the stability of protein interaction networks in cancer and neurological disorders". 

## 1 Lọc PPI

### 1.1 Nhập thư viện

In [2]:
import numpy as np
import pandas as pd

### 1.2 Đọc dữ liệu PPI, uniprot

PPI.csv (protein- protein interation): gồm hai cột, mỗi cột là một dãy các protein người ở dạng ID, mỗi dòng là các tương tác giữa các protein với nhau. <br>
uniprot-brca1-filtered-organism__Homo+sapiens+(Human)+[9606]_.csv: cột Entry chứa ID của protein, cột Gene names chứa tên của gen tương ứng với protein đó.

In [88]:
ppi = pd.read_csv("D:/Data Science/Data Science Core/Documents/Data Mining/Data/PPI.csv")
ppi.head()

,GeneA,GeneB
0,A0AV47,A0AV47
1,A0FGR8,A0FGR9
2,A5PKW4,A5PKW4
3,A8K0Z3,B3KMC4
4,A8MVA2,B0QYK1


In [82]:
uniprot = pd.read_csv("D:/Data Science/Data Science Core/Documents/Data Mining/Project/uniprot-filtered-organism__Homo+sapiens+(Human)+(9606)_+AND+review--.csv")
uniprot.head()

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,Q96IY4,CBPB2_HUMAN,reviewed,Carboxypeptidase B2 (EC 3.4.17.20) (Carboxypep...,CPB2,Homo sapiens (Human),423
1,P22362,CCL1_HUMAN,reviewed,C-C motif chemokine 1 (Small-inducible cytokin...,CCL1 SCYA1,Homo sapiens (Human),96
2,Q8NCR9,CLRN3_HUMAN,reviewed,Clarin-3 (Transmembrane protein 12) (Usher syn...,CLRN3 TMEM12 USH3AL1,Homo sapiens (Human),226
3,Q8IUK8,CBLN2_HUMAN,reviewed,Cerebellin-2,CBLN2 UNQ1892/PRO4338,Homo sapiens (Human),224
4,Q9BX69,CARD6_HUMAN,reviewed,Caspase recruitment domain-containing protein 6,CARD6,Homo sapiens (Human),1037


### 1.3 Lọc PPI

Số lượng tương tác giữa các protein

In [80]:
ppi.shape[0]

166776

#### 1.3.1 Lọc bỏ các tương tác giữa protein với chính nó.

In [92]:
ppi_self_drop = ppi[ppi["GeneA"] != ppi["GeneB"]]
ppi_self_drop.shape[0]

165797

In [145]:
ppi_self_drop.head()

,GeneA,GeneB
1,A0FGR8,A0FGR9
3,A8K0Z3,B3KMC4
4,A8MVA2,B0QYK1
6,B0QYK1,C9JC66
7,B1GXI8,B4DV56


#### 1.3.2 Kiểm tra sự dư thừa dữ liệu

Ta cần kiểm tra xem có tồn tại các liên kết giống nhau hay không, các liên kết giống nhau trong dữ liệu PPI xuất hiện nếu với một dòng (GeneA, GeneB) tồn tại một dòng khác (GeneB, GeneA).

In [146]:
# Tạo một DataFrame mới bằng cách đảo hai cột của ppi_self_drop cho nhau
ppi_inverse = pd.DataFrame({"GeneA":ppi_self_drop["GeneB"], "GeneB":ppi_self_drop["GeneA"]})
ppi_inverse.head()

,GeneA,GeneB
1,A0FGR9,A0FGR8
3,B3KMC4,A8K0Z3
4,B0QYK1,A8MVA2
6,C9JC66,B0QYK1
7,B4DV56,B1GXI8


In [148]:
# Nối hai DataFrame ppi_self_drop với ppi_inverse 
ppi_concat = pd.concat([ppi_self_drop, ppi_inverse])
ppi_concat.shape[0]

331594

In [152]:
# Chọn các hàng giống nhau
duplicate_RowsDF = ppi_concat[ppi_concat.duplicated()]
 
print("Các hàng giống nhau là:")
print(duplicate_RowsDF)

Các hàng giống nhau là:
Empty DataFrame
Columns: [GeneA, GeneB]
Index: []


Như vậy không tồn tại các liên kết dư thừa trong dữ liệu ppi_self_drop.

#### 1.3.3 Lọc các protein không có trong uniprot

Lọc bỏ các tương tác protein mà các protein không có trong "uniprot-brca1-filtered-organism__Homo+sapiens+(Human)+[9606]_.csv"

In [160]:
# Các protein id trong từng cột GeneA và GeneB của ppi_self_drop
Gene_A = np.unique(ppi_self_drop["GeneA"])
Gene_B = np.unique(ppi_self_drop["GeneB"])
print(Gene_A)
print(Gene_B)

['A0FGR8' 'A8K0Z3' 'A8MVA2' ... 'Q9Y6M7' 'Q9Y6Q9' 'Q9Y6X2']
['A0AUZ9' 'A0AV47' 'A0AV96' ... 'Q9ZPE7' 'Q9ZV85' 'Q9ZVX4']


In [172]:
# Các protein id trong dữ liệu ppi_self_drop
Gene_concat = np.concatenate((Gene_A, Gene_B))
ppi_id = np.unique(Gene_concat)
ppi_id

array(['A0AUZ9', 'A0AV47', 'A0AV96', ..., 'Q9ZPE7', 'Q9ZV85', 'Q9ZVX4'],
      dtype=object)

In [176]:
# Các protein id có trong uniprot
uniprot_id = uniprot["Entry"]
uniprot_id.head()

0    Q96IY4
1    P22362
2    Q8NCR9
3    Q8IUK8
4    Q9BX69
Name: Entry, dtype: object

In [184]:
# Các protein_id trong ppi_self_drop có trong uniprot
protein_id = list(set(ppi_id).intersection(uniprot_id))
len(protein_id)

15160

In [250]:
# Những vị trí trong cột GeneA của ppi_self_drop có trong protein_id
boolean_A = []
for x in ppi_self_drop["GeneA"]:
    boolean_A.append(x in protein_id)

In [224]:
# Những vị trí trong cột GeneB của ppi_self_drop có trong protein_id
boolean_B = []
for x in ppi_self_drop["GeneB"]:
    boolean_B.append(x in protein_id)

In [237]:
# Những vị trí ở cả hai cột GeneA và GeneB của ppio_self_drop đều có trong protein_id
boolean = []
for i in range(len(boolean_A)):
    if boolean_A[i] == True and boolean_B[i] == True:
        boolean.append(True)
    elif boolean_A[i] == True and boolean_B[i] == False:
        boolean.append(False)
    elif boolean_A[i] == False and boolean_B[i] == False:
        boolean.append(False)
    elif boolean_A[i] == False and boolean_B[i] == True:
        boolean.append(False)

Lọc các protein ở ppi_self_drop không có trong uniprot

In [241]:
ppi_filter = ppi_self_drop[boolean]

In [248]:
print("Dữ liệu PPI sau khi lọc có: %d protein và %d tương tác." %(len(protein_id),len(ppi_filter)))

Dữ liệu PPI sau khi lọc có: 15160 protein và 160320 tương tác.


## 2 Tiền xử lý dữ liệu biểu hiện Gen

In [249]:
with open ("D:/Data Science/Data Science Core/Documents/Data Mining/Data/Disease_Gene_Expression_Data_Sets/Ovary_Cancer/GSE9891_series_matrix.txt") as file:
    text = file.read()
    print(text.head())